# Capstone Project - The Battle of Neighborhoods (Week 1)  
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem
---

In this Project We will Try to Recommends the Best 5 Hotel in a City provided by User for stay and Suggesting the Nearby Places from all that hotels

**Use Case:**
- Helping Users to choose Best Hotels for stay and also suggests nearby places from all that Hotels


### The Problem:

Let's Suppose I want to Visit New York City, where the First time I'm Going to. So the First problem arrives where to Stay? Googling all the Hotels then checking Reviews, Ratings for all the Hotels and then Deciding the Hotel, then Searching for the Nearby places for all the Hotels and Many more...... That's a Very time Consuming and Tedious Task.

And another Problem arrives is How to Automate all this Task including Below Problems:

- How we get the Data of Hotels in any Given City?
- How can we Recommend Hotels to User?
- On which Features we can Rank Hotels?
- and How we find Nearby Places from that Hotels?

Yaa..This are the Problems which we will Try to solve!

### Solution:

We Can Automate all this Tasks using Machine Learning and Data Analysis  

We will Use **Foursquare API** to get the Data of Hotels in any City provided by the User, then using API we can fetch Ratings and Tips of Hotels. Based on Ratings we can Rank Hotels then Recommending Top 5 Hotels so that User can choose from that Hotels also we will search for Nearby Places for all Top 5 Hotels and Displaying to Users Which Helps them to choose the Hotel based on their Personal Favorable Nearby Places

*P.S.: Like Hotels, We Can do the Same for Restaurants Also :)*

## Data
---

First User will Enter the Name of the Place or City where He would like to Search for Hotels then taking City or Place as Input for Fetching Co-ordinates of that Location, We will use **Python's Geopy Library** to Fetch Co-ordinates then We will Generate Map of given Location to Superimpose Hotels on it.**Python's Folium Library** will be used to Generate Map

Now It's Time to use **Foursquare API!**

Using Foursquare API we will Query for Hotels near Given Location under the Radius of 10 KM, We Get JSON data as a Result then convert it into Hotels *Pandas DataFrame* having the Following Columns:

- Name: Name of the Hotel. e.g: *EVEN Hotels New York - Times Square South*
- Categories: Category of Hotel. e.g.: *Hotel, Resort*
- Address: Address of Hotel. e.g.: *321 W 35th St, New York, NY 10001, United States*
- Distance: Distance of Hotel from given City (In Meters). e.g.: *4648* 
- lat: Latitude. e.g.: *40.753467*
- lng: Longitude. e.g.: *-73.993898*
- ID: Unique ID for all Hotels. e.g.: *558dff09498e0910b7cbf9bd*

Now We can use ID of every Hotels to fetch Ratings of corresponding Hotels using Foursquare API.  
Iterating to every Hotels ID we will store the rating into new *Pandas DataFrame* with ID then Merge this new Dataframe with Hotels Dataframe.

Yeah...We Just Prepared a List of Hotels with Ratings!

Now, We will Sort the Ratings and Select Top 5 Hotels from Hotels Dataframe and Store into New Dataframe Consisting of Selected Top 5 Hotels.  
Using this Hotel's Location Co-ordinates, we will Get Nearby Places under the Radius of 1 KM from the Hotel. Quering for Nearby Places by passing co-ordinates and Radius we will get JSON data as a Result then Convert it into Nearby Places *Pandas DataFrame* having the Columns same as Hotels Dataframe.

Now, Combine both Dataframe Resulting into New Final Dataframe Consisting Data of Hotel + Nearby Places with Co-ordinates  

Finally We will Generate Map of all Nearby Places along with the 5 Hotels and make 5 clusters of Hotels + Nearby Places using **K-Means Algorithm**



-----

# Capstone Project - The Battle of Neighborhoods (Week 2)  
### Applied Data Science Capstone by IBM/Coursera

---

Let's First Import all The Required Python Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting map library

print('Folium installed')
print('All Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_0         148 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [35]:
# @Hidden_cell  

CLIENT_ID = 'BNLGDVDBSINADZPTXDGINA0FKHY3MRHTRWL1J3R33DDPZPKC' # your Foursquare ID
CLIENT_SECRET = '2MJNHWSPZQ00Z3IKO3CMKTVOLWFXH1NAMRF3V1RQBGBAI5GB' # your Foursquare Secret
VERSION = '20190713'

#### Let's again assume that I want Hotels for staying in New York City. So let's start by converting the New York City address to its latitude and longitude coordinates.

In [3]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


## 1. Search for Hotels

#### Prepaing URL for Sending GET Request

In [4]:
search_query = 'hotels'
radius = 10000 # 10 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_hotels = requests.get(url).json()
result_hotels


{'meta': {'code': 200, 'requestId': '5d2b33566c0aa5003078a8c8'},
 'response': {'venues': [{'id': '558dff09498e0910b7cbf9bd',
    'name': 'EVEN Hotels New York - Times Square South',
    'location': {'address': '321 W 35th St',
     'lat': 40.75346716759432,
     'lng': -73.99389805141669,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75346716759432,
       'lng': -73.99389805141669}],
     'distance': 4648,
     'postalCode': '10001',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['321 W 35th St',
      'New York, NY 10001',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1563112278',
    'hasPerk': False},
   {'id': '4d4711197e2e5481b91

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [5]:
# assign relevant part of JSON to hotel
hotels = result_hotels['response']['venues']

# tranform hotels into a dataframe
df_hotels = json_normalize(hotels)
df_hotels.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,558dff09498e0910b7cbf9bd,321 W 35th St,US,New York,United States,NaN,4648,"[321 W 35th St, New York, NY 10001, United Sta...","[{'label': 'display', 'lat': 40.75346716759432...",40.753467,-73.993898,10001,NY,EVEN Hotels New York - Times Square South,v-1563112278,NaN
1,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4d4711197e2e5481b9127a8f,310 W Broadway,US,New York,United States,NaN,1041,"[310 W Broadway, New York, NY 10013, United St...","[{'label': 'display', 'lat': 40.72203190438143...",40.722032,-74.004670,10013,NY,Grandlife Hotels,v-1563112278,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,572cf571cd104595c5a0ea90,46 Nevins St,US,Brooklyn,United States,NaN,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]","[{'label': 'display', 'lat': 40.687493, 'lng':...",40.687493,-73.981840,11217,NY,EVEN Hotels Brooklyn,v-1563112278,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4a6e3e75f964a5204ed41fe3,15 Gold St,US,New York,United States,at Platt St,546,"[15 Gold St (at Platt St), New York, NY 10038,...","[{'label': 'display', 'lat': 40.7078871, 'lng'...",40.707887,-74.007103,10038,NY,Gild Hall,v-1563112278,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,5841facedb6cd606fead9b04,221 E 44th St,US,New York,United States,NaN,5196,"[221 E 44th St, New York, NY 10017, United Sta...","[{'label': 'display', 'lat': 40.75183537844337...",40.751835,-73.972376,10017,NY,Even Hotels New York - Midtown East,v-1563112278,NaN


#### Define information of interest and filter dataframe

In [6]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_hotels.columns if col.startswith('location.')] + ['id']
df_hotels_filtered = df_hotels.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotels_filtered['categories'] = df_hotels_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotels_filtered.columns = [column.split('.')[-1] for column in df_hotels_filtered.columns]

pd.DataFrame(df_hotels_filtered)
df_hotels_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,New York,United States,NaN,4648,"[321 W 35th St, New York, NY 10001, United Sta...","[{'label': 'display', 'lat': 40.75346716759432...",40.753467,-73.993898,10001,NY,558dff09498e0910b7cbf9bd
1,Grandlife Hotels,Office,310 W Broadway,US,New York,United States,NaN,1041,"[310 W Broadway, New York, NY 10013, United St...","[{'label': 'display', 'lat': 40.72203190438143...",40.722032,-74.004670,10013,NY,4d4711197e2e5481b9127a8f
2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,Brooklyn,United States,NaN,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]","[{'label': 'display', 'lat': 40.687493, 'lng':...",40.687493,-73.981840,11217,NY,572cf571cd104595c5a0ea90
3,Gild Hall,Hotel,15 Gold St,US,New York,United States,at Platt St,546,"[15 Gold St (at Platt St), New York, NY 10038,...","[{'label': 'display', 'lat': 40.7078871, 'lng'...",40.707887,-74.007103,10038,NY,4a6e3e75f964a5204ed41fe3
4,Even Hotels New York - Midtown East,Hotel,221 E 44th St,US,New York,United States,NaN,5196,"[221 E 44th St, New York, NY 10017, United Sta...","[{'label': 'display', 'lat': 40.75183537844337...",40.751835,-73.972376,10017,NY,5841facedb6cd606fead9b04
5,Grand Hotels Executive Office,Residential Building (Apartment / Condo),310 W Broadway,US,New York,United States,Grand St,1043,"[310 W Broadway (Grand St), New York, NY 10013...","[{'label': 'display', 'lat': 40.722038, 'lng':...",40.722038,-74.004568,10013,NY,4be18b022740ef3b32d5210f
6,Starwood Hotels NYC,Office,75 Varick St,US,New York,United States,Grand,1184,"[75 Varick St (Grand), New York, NY 10013, Uni...","[{'label': 'display', 'lat': 40.72336996176332...",40.723370,-74.006014,10013,NY,4b6ad5d8f964a520a3e22be3
7,Hotels Cheap Booking Worldwide | أرخص حجوزات ا...,Travel Agency,E65th St,US,New York,United States,E65th St,6856,"[E65th St (E65th St), New York, NY 10065, Unit...","[{'label': 'display', 'lat': 40.76686723801730...",40.766867,-73.967246,10065,NY,5915de0bda5ede1d81c935b1
8,Leading Hotels of the World,Hotel,99 Park Ave,US,New York,United States,40th St,4795,"[99 Park Ave (40th St), New York, NY 10016, Un...","[{'label': 'display', 'lat': 40.75049469014301...",40.750495,-73.978673,10016,NY,4cadd6b9d1f8b60cf41d72c6
9,Hotels AB,None,23 E 4th St,US,New York,United States,Lafayette,1994,"[23 E 4th St (Lafayette), New York, NY 10003, ...","[{'label': 'display', 'lat': 40.727753, 'lng':...",40.727753,-73.993140,10003,NY,4c23c942a852c9288510e26c


As you have Observed in the Dataframe, **Categories** Column Containing unwanted values like Travel Agency in the 5th row So let's Clean that Rows. And one Correction is some Hotels are Categories as a **Office** but they are Hotels so we will correct them

In [7]:
df_hotels_filtered = df_hotels_filtered[df_hotels_filtered.categories.isin(['Hotel', 'Office'])]
df_hotels_filtered.categories = 'Hotel'
df_hotels_filtered.reset_index(drop=True, inplace=True)
df_hotels_filtered

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,New York,United States,NaN,4648,"[321 W 35th St, New York, NY 10001, United Sta...","[{'label': 'display', 'lat': 40.75346716759432...",40.753467,-73.993898,10001,NY,558dff09498e0910b7cbf9bd
1,Grandlife Hotels,Hotel,310 W Broadway,US,New York,United States,NaN,1041,"[310 W Broadway, New York, NY 10013, United St...","[{'label': 'display', 'lat': 40.72203190438143...",40.722032,-74.004670,10013,NY,4d4711197e2e5481b9127a8f
2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,Brooklyn,United States,NaN,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]","[{'label': 'display', 'lat': 40.687493, 'lng':...",40.687493,-73.981840,11217,NY,572cf571cd104595c5a0ea90
3,Gild Hall,Hotel,15 Gold St,US,New York,United States,at Platt St,546,"[15 Gold St (at Platt St), New York, NY 10038,...","[{'label': 'display', 'lat': 40.7078871, 'lng'...",40.707887,-74.007103,10038,NY,4a6e3e75f964a5204ed41fe3
4,Even Hotels New York - Midtown East,Hotel,221 E 44th St,US,New York,United States,NaN,5196,"[221 E 44th St, New York, NY 10017, United Sta...","[{'label': 'display', 'lat': 40.75183537844337...",40.751835,-73.972376,10017,NY,5841facedb6cd606fead9b04
5,Starwood Hotels NYC,Hotel,75 Varick St,US,New York,United States,Grand,1184,"[75 Varick St (Grand), New York, NY 10013, Uni...","[{'label': 'display', 'lat': 40.72336996176332...",40.723370,-74.006014,10013,NY,4b6ad5d8f964a520a3e22be3
6,Leading Hotels of the World,Hotel,99 Park Ave,US,New York,United States,40th St,4795,"[99 Park Ave (40th St), New York, NY 10016, Un...","[{'label': 'display', 'lat': 40.75049469014301...",40.750495,-73.978673,10016,NY,4cadd6b9d1f8b60cf41d72c6
7,Thompson Hotels Corp. HQ,Hotel,54 Thompson St,US,New York,United States,NaN,1246,"[54 Thompson St, New York, NY 10012, United St...","[{'label': 'display', 'lat': 40.72375392133431...",40.723754,-74.003467,10012,NY,4be052d4cb81c9b65a68658b
8,Commune Hotels + Resorts,Hotel,120 Wooster St,US,New York,United States,Prince,1471,"[120 Wooster St (Prince), New York, NY 10012, ...","[{'label': 'display', 'lat': 40.72520364031739...",40.725204,-74.000234,10012,NY,5265a17511d2eac871c217dd
9,Le Soleil Executive Hotels,Hotel,38 W 36th St,US,New York,United States,NaN,4539,"[38 W 36th St, New York, NY 10018, United States]","[{'label': 'display', 'lat': 40.75040662233482...",40.750407,-73.985440,10018,NY,550c5e1b498ec8c459b165c6


Now Let's Drop unwanted Columns crossStreet, labeledLatlngs, postalcode, state, country, city, cc

In [8]:
df_hotels_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
df_hotels_filtered

,name,categories,address,cc,distance,formattedAddress,lat,lng,id
0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,4648,"[321 W 35th St, New York, NY 10001, United Sta...",40.753467,-73.993898,558dff09498e0910b7cbf9bd
1,Grandlife Hotels,Hotel,310 W Broadway,US,1041,"[310 W Broadway, New York, NY 10013, United St...",40.722032,-74.004670,4d4711197e2e5481b9127a8f
2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]",40.687493,-73.981840,572cf571cd104595c5a0ea90
3,Gild Hall,Hotel,15 Gold St,US,546,"[15 Gold St (at Platt St), New York, NY 10038,...",40.707887,-74.007103,4a6e3e75f964a5204ed41fe3
4,Even Hotels New York - Midtown East,Hotel,221 E 44th St,US,5196,"[221 E 44th St, New York, NY 10017, United Sta...",40.751835,-73.972376,5841facedb6cd606fead9b04
5,Starwood Hotels NYC,Hotel,75 Varick St,US,1184,"[75 Varick St (Grand), New York, NY 10013, Uni...",40.723370,-74.006014,4b6ad5d8f964a520a3e22be3
6,Leading Hotels of the World,Hotel,99 Park Ave,US,4795,"[99 Park Ave (40th St), New York, NY 10016, Un...",40.750495,-73.978673,4cadd6b9d1f8b60cf41d72c6
7,Thompson Hotels Corp. HQ,Hotel,54 Thompson St,US,1246,"[54 Thompson St, New York, NY 10012, United St...",40.723754,-74.003467,4be052d4cb81c9b65a68658b
8,Commune Hotels + Resorts,Hotel,120 Wooster St,US,1471,"[120 Wooster St (Prince), New York, NY 10012, ...",40.725204,-74.000234,5265a17511d2eac871c217dd
9,Le Soleil Executive Hotels,Hotel,38 W 36th St,US,4539,"[38 W 36th St, New York, NY 10018, United States]",40.750407,-73.985440,550c5e1b498ec8c459b165c6


#### Let's visualize the Hotels on Map

In [32]:
allhotels_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the New York City

# add a red circle marker to represent the New York City
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York City',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(allhotels_map)

# add the Hotels as blue circle markers
for lat, lng, label in zip(df_hotels_filtered.lat, df_hotels_filtered.lng, df_hotels_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(allhotels_map)

# display map
allhotels_map

# Methodology

---

In this project we will direct our efforts on Searching Hotels of New York City that have Top Ratings, particularly those with Highest Number of Ratings. We will limit our analysis to area ~10km around city center.

In first step we have collected the required **data: location and type (category)** of every Hotels within 10km from New York City center. We have also get Coordinates of Hotels (From Foursquare API).

Second step in our analysis will be calculation and exploration of 'Ratings of Hotel' across different Hotels of New York - we will use maps to plot all Hotels then we will Fetch Ratings for All Hotels using Foursquare API then Filter Top 5 Hotels having Highest Ratings. Now we have 5 Hotels, We will Explore Nearby Places around Radius of ~500 M from Hotel.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders:  We will Cluster Hotel and their Nearby Places using ***K-means* Clustering** 

# Analysis

---

Let's perform some basic explanatory data analysis and derive some additional info from our Hotels data. First, let's Fetch Ratings for All Hotels

## 2. Fetch Ratings for All Hotels

Using For Loop we will Iterate to every Hotels and Fetch Corresponding Ratings and then Store it in new dataframe *ratings* 

In [11]:
ratings = pd.DataFrame()
list1 = pd.Series([]) 
list2 = pd.Series([]) 
for i in range(len(df_hotels_filtered.id)):
    hotel_id = df_hotels_filtered.id[i] # ID of Hotels
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(hotel_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #Let's Check for Ratings, If Available then store it and if not, then store 0
    try:
        list1[i] = result['response']['venue']['rating'] 
        list2[i] = df_hotels_filtered.id[i]
    except:
        list1[i] = 0
        list2[i] = df_hotels_filtered.id[i]
         
        
ratings.insert(0, "ID", list2, allow_duplicates=True)
ratings.insert(1, "Ratings", list1, allow_duplicates=True)
ratings

,ID,Ratings
0,558dff09498e0910b7cbf9bd,9.0
1,4d4711197e2e5481b9127a8f,0.0
2,572cf571cd104595c5a0ea90,8.2
3,4a6e3e75f964a5204ed41fe3,8.0
4,5841facedb6cd606fead9b04,7.5
5,4b6ad5d8f964a520a3e22be3,0.0
6,4cadd6b9d1f8b60cf41d72c6,0.0
7,4be052d4cb81c9b65a68658b,0.0
8,5265a17511d2eac871c217dd,0.0
9,550c5e1b498ec8c459b165c6,6.3


In [12]:
#merge both dataframes based on id
ratings.columns = ['id', 'Ratings']
df_hotels_filtered = pd.merge(df_hotels_filtered,
                 ratings,
                 on='id')
df_hotels_filtered

,name,categories,address,cc,distance,formattedAddress,lat,lng,id,Ratings
0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,4648,"[321 W 35th St, New York, NY 10001, United Sta...",40.753467,-73.993898,558dff09498e0910b7cbf9bd,9.0
1,Grandlife Hotels,Hotel,310 W Broadway,US,1041,"[310 W Broadway, New York, NY 10013, United St...",40.722032,-74.004670,4d4711197e2e5481b9127a8f,0.0
2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]",40.687493,-73.981840,572cf571cd104595c5a0ea90,8.2
3,Gild Hall,Hotel,15 Gold St,US,546,"[15 Gold St (at Platt St), New York, NY 10038,...",40.707887,-74.007103,4a6e3e75f964a5204ed41fe3,8.0
4,Even Hotels New York - Midtown East,Hotel,221 E 44th St,US,5196,"[221 E 44th St, New York, NY 10017, United Sta...",40.751835,-73.972376,5841facedb6cd606fead9b04,7.5
5,Starwood Hotels NYC,Hotel,75 Varick St,US,1184,"[75 Varick St (Grand), New York, NY 10013, Uni...",40.723370,-74.006014,4b6ad5d8f964a520a3e22be3,0.0
6,Leading Hotels of the World,Hotel,99 Park Ave,US,4795,"[99 Park Ave (40th St), New York, NY 10016, Un...",40.750495,-73.978673,4cadd6b9d1f8b60cf41d72c6,0.0
7,Thompson Hotels Corp. HQ,Hotel,54 Thompson St,US,1246,"[54 Thompson St, New York, NY 10012, United St...",40.723754,-74.003467,4be052d4cb81c9b65a68658b,0.0
8,Commune Hotels + Resorts,Hotel,120 Wooster St,US,1471,"[120 Wooster St (Prince), New York, NY 10012, ...",40.725204,-74.000234,5265a17511d2eac871c217dd,0.0
9,Le Soleil Executive Hotels,Hotel,38 W 36th St,US,4539,"[38 W 36th St, New York, NY 10018, United States]",40.750407,-73.985440,550c5e1b498ec8c459b165c6,6.3


#### Lets Remove Hotels with No Ratings 

In [13]:
df_hotels_filtered = df_hotels_filtered.set_index("Ratings")
df_hotels_filtered = df_hotels_filtered.drop(0.0, axis=0) # Delete all rows with Ratings 0
df_hotels_filtered

,name,categories,address,cc,distance,formattedAddress,lat,lng,id
Ratings,,,,,,,,,
9.0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,4648,"[321 W 35th St, New York, NY 10001, United Sta...",40.753467,-73.993898,558dff09498e0910b7cbf9bd
8.2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]",40.687493,-73.981840,572cf571cd104595c5a0ea90
8.0,Gild Hall,Hotel,15 Gold St,US,546,"[15 Gold St (at Platt St), New York, NY 10038,...",40.707887,-74.007103,4a6e3e75f964a5204ed41fe3
7.5,Even Hotels New York - Midtown East,Hotel,221 E 44th St,US,5196,"[221 E 44th St, New York, NY 10017, United Sta...",40.751835,-73.972376,5841facedb6cd606fead9b04
6.3,Le Soleil Executive Hotels,Hotel,38 W 36th St,US,4539,"[38 W 36th St, New York, NY 10018, United States]",40.750407,-73.985440,550c5e1b498ec8c459b165c6
8.5,Hyatt Regency Jersey City,Hotel,2 Exchange Pl,US,2261,"[2 Exchange Pl, Jersey City, NJ 07302, United ...",40.717172,-74.032163,4a86e69ff964a5208a0120e3
7.5,Hotel Stanford,Hotel,43 W 32nd St,US,4229,"[43 W 32nd St (at Broadway), New York, NY 1000...",40.748127,-73.987805,4b061faff964a520ece822e3
8.9,Sofitel New York,Hotel,45 W 44th St,US,5211,"[45 W 44th St (at 6th Ave), New York, NY 10036...",40.755787,-73.981762,4b4e71caf964a520f8ed26e3
8.5,Hotel Plaza Athénée,Hotel,37 E 64th St,US,6788,"[37 E 64th St (btw Madison & Park), New York, ...",40.766637,-73.968385,4ae5a3fbf964a52034a121e3


In [14]:
#Sorting the Ratings
df_hotels_filtered.sort_values('Ratings', ascending=False, inplace=True) 

#Make new dataframe for top5 hotels
df_top5_hotels = df_hotels_filtered.head(5)
df_top5_hotels

,name,categories,address,cc,distance,formattedAddress,lat,lng,id
Ratings,,,,,,,,,
9.0,EVEN Hotels New York - Times Square South,Hotel,321 W 35th St,US,4648,"[321 W 35th St, New York, NY 10001, United Sta...",40.753467,-73.993898,558dff09498e0910b7cbf9bd
8.9,Sofitel New York,Hotel,45 W 44th St,US,5211,"[45 W 44th St (at 6th Ave), New York, NY 10036...",40.755787,-73.981762,4b4e71caf964a520f8ed26e3
8.5,Hyatt Regency Jersey City,Hotel,2 Exchange Pl,US,2261,"[2 Exchange Pl, Jersey City, NJ 07302, United ...",40.717172,-74.032163,4a86e69ff964a5208a0120e3
8.5,Hotel Plaza Athénée,Hotel,37 E 64th St,US,6788,"[37 E 64th St (btw Madison & Park), New York, ...",40.766637,-73.968385,4ae5a3fbf964a52034a121e3
8.2,EVEN Hotels Brooklyn,Hotel,46 Nevins St,US,3471,"[46 Nevins St, Brooklyn, NY 11217, United States]",40.687493,-73.981840,572cf571cd104595c5a0ea90


## 3. Search for Nearby Places

Now we are going to do many steps in just one loop!  
First, we have created blank Dictionary 'd' to store multiple dataframes, Each Dataframe contains 10 Nearby Places under the radius of 500 Meters from the Hotel.  
Then, using **For Loop** we can iterate to Every Hotel's Co-ordinates and search for Nearby Places. then Finally filtering Columns we will store Dataframe in Dictionary  

In [15]:
d = {}
for i in range(len(df_top5_hotels)):
    d[i] = pd.DataFrame()
    lat = df_top5_hotels['lat'].iloc[i]
    lng = df_top5_hotels['lng'].iloc[i]
    LIMIT=10
    radius = 500
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items) # flatten JSON
      
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    df = dataframe.loc[:, filtered_columns]
    
    # filter the category for each row
    df['venue.categories'] = df.apply(get_category_type, axis=1)
    
    # clean columns
    df.columns = [col.split('.')[-1] for col in df.columns]
    
    d[i] = df

In [16]:
#Extracting Dataframes from 'd' and storing into new dataframe for each hotels
df_hotel1 = d[0]
df_hotel2 = d[1]
df_hotel3 = d[2]
df_hotel4 = d[3]
df_hotel5 = d[4]

In [17]:
df_hotel4

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Daniel,French Restaurant,60 E 65th St,US,New York,United States,btwn Park & Madison Ave,"[60 E 65th St (btwn Park & Madison Ave), New Y...","[{'label': 'display', 'lat': 40.76682918827320...",40.766829,-73.967579,NaN,10065,NY,3fd66200f964a520dee71ee3
1,CHANEL Boutique,Boutique,737 Madison Ave,US,New York,United States,64th Street,"[737 Madison Ave (64th Street), New York, NY 1...","[{'label': 'display', 'lat': 40.76682426216792...",40.766824,-73.968836,NaN,10065,NY,4b7083c4f964a520341f2de3
2,Kate Spade New York Flagship,Women's Store,789 Madison Ave,US,New York,United States,NaN,"[789 Madison Ave, New York, NY 10065, United S...","[{'label': 'display', 'lat': 40.76845870906729...",40.768459,-73.967843,NaN,10065,NY,516338d9e4b002d220cd5222
3,Hermès,Boutique,691 Madison Ave,US,New York,United States,at E 62nd St,"[691 Madison Ave (at E 62nd St), New York, NY ...","[{'label': 'display', 'lat': 40.765489333414, ...",40.765489,-73.969987,NaN,10065,NY,4b69ab96f964a52068ac2be3
4,Tony Dragon's Grille,Food Truck,62nd St And Madison Ave (Upper East Side),US,New York,United States,NaN,"[62nd St And Madison Ave (Upper East Side), Ne...","[{'label': 'display', 'lat': 40.76533495033453...",40.765335,-73.970396,NaN,NaN,NY,4a6e2000f964a520fcd31fe3
5,Rouge Tomate Cart,Food Truck,5 Avenue A,US,New York,United States,at E 64th St,"[5 Avenue A (at E 64th St), New York, NY 10009...","[{'label': 'display', 'lat': 40.7675946068997,...",40.767595,-73.970821,Central Park,10009,NY,4e0e02c28877936cb4011891
6,Club Macanudo,Lounge,26 E 63rd St,US,New York,United States,NaN,"[26 E 63rd St, New York, NY 10065, United States]","[{'label': 'display', 'lat': 40.76588925770119...",40.765889,-73.969360,NaN,10065,NY,4a8f2259f964a520171420e3
7,Park Avenue Armory,Event Space,643 Park Ave,US,New York,United States,btwn E 66th & E 67th St,"[643 Park Ave (btwn E 66th & E 67th St), New Y...","[{'label': 'display', 'lat': 40.76715891950937...",40.767159,-73.965417,NaN,10065,NY,49d675c5f964a520d15c1fe3
8,Vaucluse,French Restaurant,100 E 63rd St,US,New York,United States,at Park Ave,"[100 E 63rd St (at Park Ave), New York, NY 100...","[{'label': 'display', 'lat': 40.7650225, 'lng'...",40.765023,-73.967604,Upper East Side,10065,NY,55dbb426498e7f12a5ba62c5
9,Hotel Plaza Athénée,Hotel,37 E 64th St,US,New York,United States,btw Madison & Park,"[37 E 64th St (btw Madison & Park), New York, ...","[{'label': 'display', 'lat': 40.76663689999999...",40.766637,-73.968385,NaN,10065,NY,4ae5a3fbf964a52034a121e3


Yeah...we have Nearby Places for All 5 Hotels! 
But wait, Observe the dataframe, Our Data Contains unwanted columns like *cc,city,country,crossStreet,address,labeledLatLngs,neighborhood,postalCode,state...* 
So, Let's Clean our Data

In [19]:
# Remove All the Unwanted Columns
df_hotel1.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel2.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel3.drop(['labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel4.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city','address', 'cc'], axis=1, inplace=True)
df_hotel5.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'address', 'cc'], axis=1, inplace=True)

**We have to Remove Special Characters from dataframe inorder to use Place names as a label on map**

In [20]:
#replace (') with blank 
df_hotel1 = df_hotel1.replace('\'','',regex=True)
df_hotel2 = df_hotel2.replace('\'','',regex=True)
df_hotel3 = df_hotel3.replace('\'','',regex=True)
df_hotel4 = df_hotel4.replace('\'','',regex=True)
df_hotel5 = df_hotel5.replace('\'','',regex=True)

#### Let's Visualize 5 Hotels on Map

In [28]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the New York City

# add a marker to represent Hotel1
folium.Marker(
    [df_top5_hotels['lat'].iloc[0], df_top5_hotels['lng'].iloc[0]],
    popup=df_top5_hotels['name'].iloc[0],
).add_to(hotels_map)

# add the Nearby Places as blue circle markers
for lat, lng, label in zip(df_hotel1.lat, df_hotel1.lng, df_hotel1.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)
    
# add a marker to represent Hotel2
folium.Marker(
    [df_top5_hotels['lat'].iloc[1], df_top5_hotels['lng'].iloc[1]],
    popup=df_top5_hotels['name'].iloc[1],
).add_to(hotels_map)

# add the Nearby Places as green circle markers
for lat1, lng1, label1 in zip(df_hotel2.lat, df_hotel2.lng, df_hotel2.name):
    folium.features.CircleMarker(
        [lat1, lng1],
        radius=5,
        color='green',
        popup=label1,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(hotels_map)
    
# add a marker to represent Hotel3
folium.Marker(
    [df_top5_hotels['lat'].iloc[2], df_top5_hotels['lng'].iloc[2]],
    popup=df_top5_hotels['name'].iloc[2],
).add_to(hotels_map)

# add the Nearby Places as orange circle markers
for lat, lng, label in zip(df_hotel3.lat, df_hotel3.lng, df_hotel3.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(hotels_map)
    
# add a marker to represent Hotel4
folium.Marker(
    [df_top5_hotels['lat'].iloc[3], df_top5_hotels['lng'].iloc[3]],
    popup=df_top5_hotels['name'].iloc[3],
).add_to(hotels_map)

# add the Nearby Places as purple circle markers
for lat, lng, label in zip(df_hotel4.lat, df_hotel4.lng, df_hotel4.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(hotels_map)
    
# add a marker to represent Hotel5
folium.Marker(
    [df_top5_hotels['lat'].iloc[4], df_top5_hotels['lng'].iloc[4]],
    popup=df_top5_hotels['name'].iloc[4],
).add_to(hotels_map)

# add the Nearby Places as red circle markers
for lat, lng, label in zip(df_hotel5.lat, df_hotel5.lng, df_hotel5.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

Yeah Finallyyy We have Successfully Generated Map of 5 Hotels with Nearby Places

Now Final Step! Cluster all this 5 Hotels based on Nearby Places and Hotels as a Center of Cluster

## 4. Clustering Hotels with Nearby Places using K-Means

Before Clustering Let's Combine all the Nearby places in one dataframe to generate Clusters

In [22]:
#Combine all dataframe and store it in df_nearbyplaces then reset the index values
df_nearbyplaces = pd.concat([df_hotel1, df_hotel2, df_hotel3, df_hotel4, df_hotel5], axis=0).reset_index(drop=True)
df_nearbyplaces.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,categories,formattedAddress,id,lat,lng,name,neighborhood
0,Music Venue,"[328 W 34th St (btwn 8th & 9th Ave.), New York...",4b758e78f964a520ed132ee3,40.752632,-73.994585,Music Choice,NaN
1,Hotel,"[321 W 35th St, New York, NY 10001, United Sta...",558dff09498e0910b7cbf9bd,40.753467,-73.993898,EVEN Hotels New York - Times Square South,NaN
2,Art Gallery,"[325 W 38th St. Store 1 (8th ave), New York, N...",4b872d23f964a520d1b431e3,40.755285,-73.992821,Last Rites Tattoo Theatre and Art Gallery,NaN
3,Italian Restaurant,"[456 9th Ave (btwn 35th St & 36th St), New Yor...",4dcae1b18877502200c95822,40.754388,-73.995642,Sergimmo Salumeria,NaN
4,Camera Store,"[420 9th Ave (at W 34th St), New York, NY 1000...",4adb6119f964a520a72621e3,40.753277,-73.996470,B&H Photo Video,NaN


In [23]:
from sklearn.cluster import KMeans

number_of_clusters = 5 #define number of cluster to be generated. In our case it is 5 because we have 5 hotels

loc = df_nearbyplaces[['lat', 'lng']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(loc)

labels = kmeans.labels_

In [24]:
df_nearbyplaces["Labels"] = labels #adding cluster labels to main dataframe
df_nearbyplaces.head()

,categories,formattedAddress,id,lat,lng,name,neighborhood,Labels
0,Music Venue,"[328 W 34th St (btwn 8th & 9th Ave.), New York...",4b758e78f964a520ed132ee3,40.752632,-73.994585,Music Choice,NaN,3
1,Hotel,"[321 W 35th St, New York, NY 10001, United Sta...",558dff09498e0910b7cbf9bd,40.753467,-73.993898,EVEN Hotels New York - Times Square South,NaN,3
2,Art Gallery,"[325 W 38th St. Store 1 (8th ave), New York, N...",4b872d23f964a520d1b431e3,40.755285,-73.992821,Last Rites Tattoo Theatre and Art Gallery,NaN,3
3,Italian Restaurant,"[456 9th Ave (btwn 35th St & 36th St), New Yor...",4dcae1b18877502200c95822,40.754388,-73.995642,Sergimmo Salumeria,NaN,3
4,Camera Store,"[420 9th Ave (at W 34th St), New York, NY 1000...",4adb6119f964a520a72621e3,40.753277,-73.996470,B&H Photo Video,NaN,3


In [25]:
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
df_nearbyplaces['Labels'].unique()

array([3, 4, 2, 1, 0])

### Let's Visualize Our Final Data on Map with Clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the New York City

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add Cluster circles to the map
for lon, lat in zip(cluster_centers[0], cluster_centers[1]):
    folium.CircleMarker([lon, lat], radius=50, color='yellow', fill=True, fill_opacity=0.25).add_to(map_clusters)
    folium.Marker(
    [lon, lat],
    popup='Cluster Center',
    ).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_nearbyplaces['lat'], df_nearbyplaces['lng'], df_nearbyplaces['name'], df_nearbyplaces['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Not bad! - our clusters represent groupings of Hotels with Nearby locations and cluster centers are placed nicely in the middle of the zones with location candidates.

# Results and Discussion

---

Our Analysis Shows that We Successfully Retrived All the Hotels in the New York City, and we able to Retrive Ratings of Few Hotels Only.  
Then we sorted All Hotels based on Ratings and Selected Top 5 Hotels to Recommend it to User.  
Then We Looked for Nearby Places from Selected 5 Hotels, from Data of Nearby Places we seen that We have Various Category of Places, Users can Choose Hotel based on having Their Interested Nearby Places to Visit and Enjoy. Finally We Clustered All Hotels with Nearby Places Together, So that Users can choose Area in the City, Where they would like to stay.   

As we know, Just Location and Nearby Places isn't a Good factor to Choose Hotel, Other Factors like Price, Luxury, Rooms Availability and many more Features are Necessary to Decide Hotel.

This is just Basic Exploratory Analysis we did, Further Deep Analysis can be done, Few of them are listed below.  

**Future Enhancements:**
-
- We Can Retrive Check-in and Check-out Time, Rooms Availability and Rates of Hotels from Foursquare API.
- We Can Recommend Best Nearest Restaurants From Hotel based on User Preferences Like Chinese Food, Italian Food etc.. 

# Conclusion

---

Purpose of this project was to Recommend Best Hotel in Given City with Nearby Places in order to aid Tourists in narrowing down the search for optimal location for Stay. By Getting Ratings of All Hotel from Foursquare data we have first sorted Top 5 Hotels that justify further analysis, and then generated extensive collection of Nearby Places which satisfy some basic requirements regarding favourite Places. Clustering of those Nearby Places was then performed in order to create major zones of interest (containing Nearby Places) and addresses of those zone centers were created to be used as starting points for final exploration by Users.

Final decission on optimal Hotel will be made by Users based on specific Requirements in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.

---

# Thank You!